In [14]:
import os
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

annotation_path = '../../data/train.txt'

with open(annotation_path) as f:
    lines = f.readlines()

# get image path and bounding boxes
#line = lines[21].split()
line = lines[0].split()

# load ground truth boxes
gt_boxes = np.array([np.array(list(map(int,box.split(',')))) for box in line[1:]])

# load original image
img = cv.imread(line[0])

height, width, channels = img.shape

# initialize mask with all zeros
mask = np.zeros((height, width, 3), np.uint8)

for bb in gt_boxes:
    # assume word is horizontal FIX
    # only choose horizontal words
    box_width = bb[2] - bb[0]
    box_height = bb[3] - bb[1]
    if box_width/box_height < 1.5:
        continue
    
    # choose random point between vertical edges
    mask_centroid = (bb[2] - bb[0]) * np.random.random_sample() + bb[0]
    
    # choose random width from 
    # 0.2*(width of box) and 2*(distance from centroid to closest edge)
    closest_dist = min(mask_centroid-bb[0], bb[2]-mask_centroid)
    mask_width = ((2*closest_dist) - (0.2*box_width)) * np.random.random_sample() + (0.2*box_width)
    
    # fill in mask area with white
    #white = 255
    white = [255, 255, 255]
    x_min = int(np.floor(mask_centroid - (mask_width/2)))
    y_min = bb[1]
    x_max = int(np.ceil(mask_centroid + (mask_width/2)))
    y_max = bb[3]
    
    #print(x_min, y_min, x_max, y_max)
    
    if np.min([x_min, y_min, x_max, y_max]) >= 0:
        mask[y_min:y_max, x_min:x_max] = white

# plt.imshow(mask)
# plt.title('mask')
# plt.show()
    
# https://docs.opencv.org/3.4.0/df/d3d/tutorial_py_inpainting.html
mask_gray = cv.cvtColor(mask, cv.COLOR_BGR2GRAY)
dst = cv.inpaint(img, mask_gray, 3, cv.INPAINT_TELEA)

# plt.imshow(dst)
# plt.title('dst')
# plt.show()

cv.imwrite( "./og.tiff", img )
cv.imwrite( "./out.tiff", dst )


True